# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'> Emotion Analysis with TensorFlow, Hugging Face and BERT </div></b>

![](https://img.freepik.com/free-photo/emoji-faces-social-media_53876-90190.jpg?w=900&t=st=1696607587~exp=1696608187~hmac=9977ec8f4ee3c93c853a2b075129ed04db2bb91d946658bff7bb2aeb3eac7d07)

Hi guys 😀 In this notebook, we're going to cover how to perform sentiment analysis with Hugging Face and TensorFlow. Let's get started.

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>1. Dataset Loading </div></b>

In [1]:
!pip install -q datasets

In [2]:
from datasets import load_dataset
emotions = load_dataset("dair-ai/emotion")
emotions

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/dair-ai___emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>2. Understanding the Dataset </div></b>

In [3]:
emotions["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [4]:
emotions["train"].column_names

['text', 'label']

In [5]:
emotions["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=6, names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>3. Data Preprocessing </div></b>

In [6]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [7]:
def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

In [8]:
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Padding

In [9]:
from transformers import DataCollatorWithPadding

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

## Evaluate

In [11]:
!pip install -q evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
import evaluate
accuracy = evaluate.load("accuracy")

In [13]:
import numpy as np 

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, 
                            references = labels)

# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>4. Model Training </div></b>

In [14]:
from transformers import TFAutoModelForSequenceClassification

id2label={0:"sadness",1:"joy",2:"love",3:"anger",4:"fear",5:"suprise"}
label2id={"sadness":0,"joy":1,"love":2,"anger":3,"fear":4,"suprise":5}

model=TFAutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=6, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [15]:
from huggingface_hub import notebook_login
notebook_login()

## Preparing the datasets

In [16]:
tf_train_set = model.prepare_tf_dataset(
    emotions_encoded["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator
)

tf_validation_set = model.prepare_tf_dataset(
    emotions_encoded["validation"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
import tensorflow as tf

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer)

## Setting the callbacks

In [18]:
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback

metric_callback=KerasMetricCallback(
    metric_fn=compute_metrics,
    eval_dataset = tf_validation_set
)

push_to_hub_callback=PushToHubCallback(
    output_dir = "emotion-analysis-with-distilbert",
    tokenizer = tokenizer
)

Cloning https://huggingface.co/Tirendaz/emotion-analysis-with-distilbert into local empty directory.


Download file tf_model.h5:   0%|          | 15.4k/256M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/256M [00:00<?, ?B/s]

In [19]:
model.fit(x=tf_train_set,
         validation_data=tf_validation_set,
         epochs=2,
         callbacks=[metric_callback, push_to_hub_callback])

Epoch 1/2
1000/1000 [==============================] - 145s 121ms/step - loss: 0.3930 - val_loss: 0.1628 - accuracy: 0.9340
Epoch 2/2
1000/1000 [==============================] - 89s 89ms/step - loss: 0.1302 - val_loss: 0.1400 - accuracy: 0.9340


# <b><div style='padding:15px;background-color:#850E35;color:white;border-radius:2px;font-size:110%;text-align: center'>5. Prediction </div></b>

In [20]:
custom_text = "I watched a movie yesterday. It was really awesome"

In [21]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", 
                     model = "Tirendaz/emotion-analysis-with-distilbert")

Some layers from the model checkpoint at Tirendaz/emotion-analysis-with-distilbert were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at Tirendaz/emotion-analysis-with-distilbert and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
inputs=tokenizer(custom_text, return_tensors= "tf")
logits = model(**inputs).logits
predicted_class_id=int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'joy'

**Thanks for reading! If you enjoyed this notebook, don't forget to upvote** 👍

*Let's connect* [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [X](http://x.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎

## Resource

- [NLP with Transformers](https://github.com/nlp-with-transformers/notebooks/blob/main/02_classification.ipynb)